In [ ]:
if (!dir.exists("./Functions")) {
    setwd("..")
}
source("./Functions/lecospectR.R")

In [ ]:
first_prediction_path <- system.file("", package = "raster")
second_prediction_path <- system.file("", package = "raster")
third_prediction_path <- system.file("", package = "raster")

In [ ]:
weights <- c(1.0,1.0,1.0)
target_cols <- 2:4

In [ ]:
predctions1_ras <- raster::raster(first_prediction_path)
predctions2_ras <- raster::raster(second_prediction_path)
predctions3_ras <- raster::raster(third_prediction_path)
predictions_ras <- raster::brick(list(
    predctions1_ras,
    predctions2_ras,
    predctions3_ras
))

In [ ]:
rm(predctions1_ras)
rm(predctions2_ras)
rm(predctions3_ras)
gc()

In [ ]:
prediction_df <- raster::rasterToPoints(predctions_ras)
rm(predictions_ras)
gc()

# Weighting (and Christmas)
Add Confusion matrix to manuscript

Weight the model votes by PFT

First do gloabl, then refine

In [ ]:
vote <- function(predictions, weights = NULL){
    # get the weights from the user, or default to 1s
    updated_weights <- weights
    if(is.null(weights)){
        updated_weights <- rep(1.0, times = length(predictions))
    }
    
    classes <- unique(predictions %>% as.character())
    vote_aggregator <- list()
    for(class in classes){
        vote_aggregator[[class]] <- 0
    }

    for(index in seq_along(predictions)){
        vote_aggregator[[predctions[[index]]]] <- vote_aggregator[[predctions[[index]]]] + weights[[index]]
    }


    max_class <- NULL
    max_vote <- -1
    for(class in classes){
        if(vote_aggregator[[class]] > max_vote){
            max_vote <- vote_aggregator[[class]]
            max_class <- class
        }
    }

    return(class)
}

In [ ]:
get_majority_predictions <- function(prediction_df, vote_weights = NULL){
    predictions <- character(length = nrow(prediction_df))

    for(row_index in seq(nrow(prediction_df))){
        row <- vote(prediction_df[row_index], weights = vote_weights) %>% as.character()
        predictions[[row_index]] <- vote(row, weights = vote_weights)
    }

    return(predictions)
    
}

In [ ]:
prediction_fac <- get_majority_predictions(
    prediction_df = prediction_df[,target_cols], 
    vote_weights = weights)